In [2]:
import mysql.connector
import pandas as pd

ModuleNotFoundError: No module named 'mysql'

# Connect to database

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="msu e-sadza"
)

print ("msu e-sadza")

# Register students into E-Sadza system

In [ ]:
print("Please create E-Sadza account")
reg_number = input("Enter Reg Number")
username = input("Enter username")
password = input ("Enter password")
mycursor = mydb.cursor()
sql = "INSERT INTO login (username, password,reg_number) VALUES (%s, %s,%s)"
val = (username,password,reg_number)
mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

# Add money to student Account

In [ ]:
print("Add amount to account")
reg_number = input("Enter Reg Number")
mycursor = mydb.cursor()

sql = "SELECT * FROM login WHERE reg_number = %s"
val = (reg_number,)
mycursor.execute(sql, val)
myresult = mycursor.fetchall()

if len(myresult)==0:
    print("Student is not registered")
else:
    amount = float(input("Enter amount"))
    sql = "SELECT amount FROM account WHERE reg_number = %s"
    val = (reg_number,)
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()
    if len(myresult)==0:
        sql = "INSERT INTO account (reg_number, amount) VALUES (%s, %s)"
        val = (reg_number,amount)
        mycursor.execute(sql, val)
        mydb.commit()
    else:
        amount = amount + myresult[0][0]
        sql = "UPDATE account set amount = %s where reg_number = %s"
        val = (amount,reg_number)
        mycursor.execute(sql, val)
        mydb.commit()


# Get all products from database

In [ ]:
products = []
mycursor = mydb.cursor()
sql = "SELECT * FROM product_list"
mycursor.execute(sql)
myresult = mycursor.fetchall()
for x in myresult:
    products.append(list(x))
df = pd.DataFrame(products,columns=["ID","Product","Price/$"])
df

# Buy from Canteen

In [ ]:
print("Enter Product ID to cart,press 0 to stop")
reg_number = int(input("Enter Reg Number"))
sql = "SELECT amount FROM account WHERE reg_number = %s"
val = (reg_number,)
mycursor.execute(sql, val)
myresult = mycursor.fetchall()
total_amount = 0
if len(myresult)==0:
    print("Student not registered")
else:
    total_amount = myresult[0][0]


cart_product = int(input("Enter product ID"))
cart= []
while cart_product != 0:
    if cart_product not in list(df.ID):
        print("Product ID does not exist")
    elif cart_product in cart:
        print("Product already in cart")
    else:
        sql = "SELECT price FROM product_list WHERE id = %s"
        val = (cart_product,)
        mycursor.execute(sql, val)
        myresult = mycursor.fetchall()
        cart.append(myresult[0][0])
    cart_product = int(input("Enter product ID"))

if sum(cart) > total_amount:
    print("Not enough funds in account")
else:
    amount = total_amount - sum(cart)
    sql = "UPDATE account set amount = %s where reg_number = %s"
    val = (amount,reg_number)
    mycursor.execute(sql, val)
    mydb.commit()

# Check Student Balance

In [ ]:
reg_number = input("Enter Reg Number")
sql = "SELECT amount FROM account WHERE reg_number = %s"
val = (reg_number,)
mycursor.execute(sql, val)
myresult = mycursor.fetchall()
print(myresult[0][0])

# Check Student Details

In [ ]:
reg_number = input("Enter Reg Number")
sql = "SELECT * FROM login WHERE reg_number = %s"
val = (reg_number,)
mycursor.execute(sql, val)
myresult = mycursor.fetchall()
print(myresult[0])